# Medallion Architecture

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/02_core_patterns/medallion_architecture/quickstart_tutorial.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/02_core_patterns/medallion_architecture/quickstart_tutorial.ipynb)

## Business Scenario

A CRM export has duplicates, invalid emails, and bad spend values. You need a Bronze layer to capture raw data and a Silver layer to enforce quality and business rules.

## Value Proposition

- Separate raw ingestion from validated outputs
- Make failures visible without stopping ingestion
- Create a repeatable Bronze -> Silver pipeline

---

## Goals

1. Run Bronze ingestion
2. Apply Silver validation
3. Materialize clean outputs


## 1. Setup
We'll import the `DataProcessor` and a few lightweight helpers. LakeLogic will auto-select the best available engine on your machine.


In [ ]:
from pathlib import Path
import csv
import shutil
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "02_core_patterns" / "medallion_architecture" / "contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

data_path = BASE / "data" / "crm_export.csv"

RESET_OUTPUTS = True
# Set to False to keep existing outputs on reruns.

def reset_outputs():
    targets = [
        BASE / "data" / "bronze" / "bronze_customers.parquet",
        BASE / "data" / "silver" / "silver_customers.parquet",
    ]
    for target in targets:
        if target.exists():
            if target.is_dir():
                shutil.rmtree(target)
            else:
                target.unlink()

def read_csv_rows(path, limit=5):
    rows = []
    with open(path, newline="", encoding="utf-8") as handle:
        reader = csv.DictReader(handle)
        for idx, row in enumerate(reader):
            rows.append(row)
            if idx + 1 >= limit:
                break
    return rows

def preview_rows(rows, title=None):
    if title:
        print(title)
    if not rows:
        print("<no rows>")
        return
    for row in rows:
        print(row)

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "show"):
        try:
            df.show(limit, truncate=False)
            return
        except Exception:
            pass
    if hasattr(df, "head"):
        try:
            head = df.head(limit)
            print(head)
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            sample = df.limit(limit)
            if hasattr(sample, "to_dicts"):
                for row in sample.to_dicts():
                    print(row)
                return
            print(sample)
            return
        except Exception:
            pass
    try:
        if isinstance(df, list):
            for row in df[:limit]:
                print(row)
            return
    except Exception:
        pass
    print(df)

def print_run_summary(proc, title=None):
    if title:
        print(title)
    report = proc.last_report or {}
    counts = report.get("counts") or {}
    summary = {
        "stage": report.get("stage"),
        "source": counts.get("source"),
        "total": counts.get("total"),
        "good": counts.get("good"),
        "quarantined": counts.get("quarantined"),
    }
    print(summary)


## 2. Peek at the Messy Data
Let's look at the raw contents of `data/crm_export.csv`. Note the issues:
- Customer `104` appearing twice (duplicate).
- Customer `102` having an invalid email format.
- Customer `103` missing an email entirely.
- Customer `105` having negative spend.
- Customer `107` having non-numeric spend data ("high").

In [ ]:
if not data_path.exists():
    raise FileNotFoundError(f"Missing input file: {data_path.resolve()}")

rows = read_csv_rows(data_path, limit=8)
preview_rows(rows, "RAW DATA:")


## 3. The Multi-Stage Contract
LakeLogic allows you to run the **same contract** in different "stages". 

Take a moment to open `contract.yaml` in your editor. You'll see the **Stages** defined at the top:
- **Bronze Stage**: Ingests raw data using a glob pattern (`data/crm_*.csv`), deduplicates, and renames columns.
- **Silver Stage (Default)**: Consumes the Bronze Parquet output, renames fields, deduplicates, and enforces strict business quality rules.

Run logs capture metadata like `max_source_mtime` and `source_files_json` for incremental watermarking and are stored using the contract metadata settings.

By putting stages at the top, the contract becomes a self-documenting pipeline blueprint.


## 4. Execute Step 1: Bronze Ingestion
Notice how we don't need to specify a path here! `DataProcessor` automatically pulls the correct path from the `bronze` stage definition.

In [ ]:
if RESET_OUTPUTS:
    reset_outputs()

bronze_proc = DataProcessor(contract=contract_path, stage="bronze")
bronze_result = bronze_proc.run_source()
bronze_proc.materialize(bronze_result.good, bronze_result.bad)

print_run_summary(bronze_proc, "Bronze run summary:")
preview_frame(bronze_result.good, "BRONZE OUTPUT (sample):")


## 5. Execute Step 2: Silver Quality Gate
Now we run the default (Silver) stage. It uses the `source.path` defined for the Silver layer, which points to the files we just created in Bronze.

In [ ]:
silver_proc = DataProcessor(contract=contract_path)
silver_result = silver_proc.run_source()
silver_proc.materialize(silver_result.good, silver_result.bad)

print_run_summary(silver_proc, "Silver run summary:")


## 6. Inspect Results

### The 'Good' Data (Silver Table)
Only records that passed **every** rule made it here. Notice that types are now correct and business logic is satisfied.

In [ ]:
preview_frame(silver_result.good, "PRODUCTION-READY RECORDS (sample):")


### The 'Bad' Data (Quarantine)
Records that failed are isolated with detailed reasons in `_lakelogic_errors`.

In [ ]:
preview_frame(silver_result.bad, "QUARANTINED RECORDS (sample):")


In [ ]:
silver_output = BASE / "data" / "silver" / "silver_customers.parquet"
print(f"Silver output written to: {silver_output.resolve()}")
print(f"Exists: {silver_output.exists()}")


### Run Logs
Below we print the latest run report captured during this session.


In [ ]:
print("Latest run report:")
report = silver_proc.last_report or {}
print({"stage": report.get("stage"), "counts": report.get("counts")})


## 7. Next Steps
Congratulations! You've just built a production-quality data gate.

**Try this:**
1. Edit `contract.yaml` to change the `bronze` stage path to a missing file and see how it fails gracefully.
2. Add a new CSV to the `data/` folder and watch the `bronze` stage pick it up automatically via the glob pattern!
